# DeepFake Detection Model
**Install the necessary packages**


In [3]:
# !pip install moviepy
# !pip install autokeras
# !pip install --upgrade keras_nlp
# # Fix for the plotting in Kaggle 
# !pip install pydot
# !apt-get install -y graphviz
# !pip install graphviz

**Import the required packages**

In [1]:
import tensorflow as tf
import os
import glob
import cv2
import shutil
# import torchaudio
import numpy as np
import torch
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from tensorflow.keras.layers import *
from keras.models import Model
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import to_categorical, plot_model
from tensorflow.keras.optimizers import Adam
# import autokeras as ak
from keras.models import save_model
from moviepy.video.io.VideoFileClip import VideoFileClip
import librosa
from sklearn.model_selection import train_test_split
import h5py
import shutil

In [2]:

# latent_dim = 10
# image_shape = (64, 64, 3)
# output_shape = (64, 64, 3)
# batch_size = 64
# epochs = 10000
# save_interval = 1000

# # Build the Generator model
# def build_generator(latent_dim, output_shape):
#     generator_input = Input(shape=(latent_dim,))
#     x = Dense(64)(generator_input)
#     x = LeakyReLU()(x)
#     x = Dense(128)(x)
#     x = LeakyReLU()(x)

#     # Calculate the number of units in the Dense layer for the generator output
#     num_units = output_shape[0] * output_shape[1] * output_shape[2]

#     # Generator output with a Dense layer and reshape
#     generated_data = Dense(num_units, activation='tanh')(x)
#     generated_data = Reshape(output_shape)(generated_data)

#     generator = Model(generator_input, generated_data)
#     return generator

# # Build the Discriminator model
# def build_discriminator(input_shape):
#     discriminator_input = Input(shape=input_shape)
#     x = Dense(128)(discriminator_input)
#     x = LeakyReLU()(x)
#     x = Dense(64)(x)
#     x = LeakyReLU()(x)
#     validity = Dense(1, activation='sigmoid')(x)
#     discriminator = Model(discriminator_input, validity)
#     return discriminator

# # Build the GAN model
# def build_gan(generator, discriminator):
#     discriminator.trainable = False
#     gan_input = Input(shape=(latent_dim,))
#     generated_data = generator(gan_input)
#     gan_output = discriminator(generated_data)
#     gan = Model(gan_input, gan_output)
#     return gan

# # generator = build_generator(latent_dim, output_shape)
# # discriminator = build_discriminator(output_shape)
# # gan = build_gan(generator, discriminator)

# # Print the model summaries

# # Plot the model architectures

# generator = build_generator(latent_dim, output_shape)
# generator.compile(optimizer=Adam(0.0002, 0.5), loss='binary_crossentropy')  # Compile generator

# discriminator = build_discriminator(output_shape)
# discriminator.compile(optimizer=Adam(0.0002, 0.5), loss='binary_crossentropy', metrics=['accuracy'])

# gan = build_gan(generator, discriminator)
# gan.compile(optimizer=Adam(0.0002, 0.5), loss='binary_crossentropy')

# generator.summary()
# discriminator.summary()
# gan.summary()

In [3]:
# def train(epochs, batch_size, save_interval):
#     valid = np.ones((batch_size, 1))
#     fake = np.zeros((batch_size, 1))

#     for epoch in range(epochs):
#         # Train Discriminator
#         idx = np.random.randint(0, X_train.shape[0], batch_size)
#         real_imgs = X_train[idx]

#         noise = np.random.normal(0, 1, (batch_size, latent_dim))
#         gen_imgs = generator.predict(noise)

#         d_loss_real = discriminator.train_on_batch(real_imgs, valid)
#         d_loss_fake = discriminator.train_on_batch(gen_imgs, fake)
#         d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

#         # Train Generator
#         noise = np.random.normal(0, 1, (batch_size, latent_dim))
#         g_loss = gan.train_on_batch(noise, valid)

#         # Print the progress
#         print(f"{epoch} [D loss: {d_loss[0]} | D acc.: {100*d_loss[1]}] [G loss: {g_loss}]")

#         # Save generated images at intervals
#         if epoch % save_interval == 0:
#             save_imgs(epoch)

# # Save generated images function
# def save_imgs(epoch):
#     r, c = 5, 5
#     noise = np.random.normal(0, 1, (r * c, latent_dim))
#     gen_imgs = generator.predict(noise)

#     gen_imgs = 0.5 * gen_imgs + 0.5  # Rescale to [0, 1]

#     fig, axs = plt.subplots(r, c)
#     cnt = 0
#     for i in range(r):
#         for j in range(c):
#             axs[i, j].imshow(gen_imgs[cnt, :, :, 0])
#             axs[i, j].axis('off')
#             cnt += 1
#     fig.savefig(f"images/gan_{epoch}.png")
#     plt.close()


In [4]:
def multimodal_fusion(images, num_modalities):
    num_samples, num_frames, height, width, channels = images.shape

    # Create additional modalities by converting images to grayscale
    additional_modalities = []
    for i in range(1, num_modalities):
        modified_images = []
        for frame in range(num_frames):
            grayscale_frame = np.mean(images[:, frame], axis=-1, keepdims=True)
            modified_images.append(grayscale_frame)
        additional_modalities.append(np.array(modified_images))

    # Tile the additional modalities to match the number of frames
    tiled_modalities = [modalities[:, np.newaxis] for modalities in additional_modalities]
    tiled_modalities = [np.tile(modalities, (1, num_frames, 1, 1, 1)) for modalities in tiled_modalities]

    # Concatenate the original images and additional modalities along the channel axis
    fused_data = np.concatenate((images, *tiled_modalities), axis=-1)
    return fused_data

**Create additional modalities**

In [5]:
batch_size = 40
num_modalities = 1 #Add GrayScale with RGB modality

# Load the data from the HDF5 file
h5_file = h5py.File('data.h5', 'r')
num_samples = len(h5_file['X_images_train'])
y_train = h5_file['y_train'][:]

multimodal_data = []

for batch_start in range(0, num_samples, batch_size):
    batch_end = min(batch_start + batch_size, num_samples)
    batch_X_images_train = h5_file['X_images_train'][batch_start:batch_end]
    #batch_X_images_test = h5_file['X_images_test'][batch_start:batch_end]

    # Perform multimodal fusion on the current batch
    fused_batch_X_train = multimodal_fusion(batch_X_images_train, num_modalities)
    #fused_batch_X_test = multimodal_fusion(batch_X_images_test, num_modalities)
    multimodal_data.append(fused_batch_X_train)
    print("Fused batch X_train shape:", fused_batch_X_train.shape)
    #print("Fused batch X_test shape:", fused_batch_X_test.shape)

h5_file.close()

Fused batch X_train shape: (40, 10, 64, 64, 3)
Fused batch X_train shape: (40, 10, 64, 64, 3)
Fused batch X_train shape: (40, 10, 64, 64, 3)
Fused batch X_train shape: (40, 10, 64, 64, 3)


In [6]:
# Stack the multimodal data
fused_X_train = np.vstack(multimodal_data)

# Get the dimensions from the shape of fused_X_train
num_samples, num_frames, height, width, channels = fused_X_train.shape

# Reshape the data to match AutoKeras expected format
reshaped_X_train = fused_X_train.reshape((-1, height, width, channels))

# Reshape again to treat each frame as a separate sample
reshaped_X_train = reshaped_X_train.reshape((-1, height, width, channels))

# Reshape y_train to match the new number of samples
reshaped_y_train = np.repeat(y_train, num_frames, axis=0)

print("reshaped_X_train shape:", reshaped_X_train.shape)
print("reshaped_y_train shape:", reshaped_y_train.shape)

X_train, X_val, y_train, y_val = train_test_split(reshaped_X_train, reshaped_y_train, test_size=0.2, random_state=42)


reshaped_X_train shape: (1600, 64, 64, 3)
reshaped_y_train shape: (1600,)


In [7]:
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, LeakyReLU, Reshape, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.datasets import cifar10  # Using CIFAR-10 dataset as an example
import matplotlib.pyplot as plt

latent_dim = 10
image_shape = (64, 64, 3)
output_shape = (64, 64, 3)
batch_size = 64
epochs = 5000
save_interval = 1000

# Build the Generator model
def build_generator(latent_dim, output_shape):
    generator_input = Input(shape=(latent_dim,))
    x = Dense(256)(generator_input)
    x = LeakyReLU()(x)
    x = Dense(512)(x)
    x = LeakyReLU()(x)
    x = Dense(1024)(x)
    x = LeakyReLU()(x)
    
    num_units = np.prod(output_shape)
    generated_data = Dense(num_units, activation='tanh')(x)
    generated_data = Reshape(output_shape)(generated_data)
    generator = Model(generator_input, generated_data)
    return generator

# Build the Discriminator model
def build_discriminator(input_shape):
    discriminator_input = Input(shape=input_shape)
    x = Flatten()(discriminator_input)
    x = Dense(512)(x)
    x = LeakyReLU()(x)
    x = Dense(256)(x)
    x = LeakyReLU()(x)
    x = Dense(1, activation='sigmoid')(x)
    discriminator = Model(discriminator_input, x)
    return discriminator

# Build the GAN model
def build_gan(generator, discriminator):
    discriminator.trainable = False
    gan_input = Input(shape=(latent_dim,))
    generated_data = generator(gan_input)
    gan_output = discriminator(generated_data)
    gan = Model(gan_input, gan_output)
    return gan

# Compile models
generator = build_generator(latent_dim, output_shape)
generator.compile(optimizer=Adam(0.0002, 0.5), loss='binary_crossentropy')  # Compile generator

discriminator = build_discriminator(output_shape)
discriminator.compile(optimizer=Adam(0.0002, 0.5), loss='binary_crossentropy', metrics=['accuracy'])

gan = build_gan(generator, discriminator)
gan.compile(optimizer=Adam(0.0002, 0.5), loss='binary_crossentropy')


# Training function
def train(epochs, batch_size, save_interval):
    valid = np.ones((batch_size, 1))
    fake = np.zeros((batch_size, 1))

    for epoch in range(epochs):
        # Train Discriminator
        idx = np.random.randint(0, X_train.shape[0], batch_size)
        real_imgs = X_train[idx]

        noise = np.random.normal(0, 1, (batch_size, latent_dim))
        gen_imgs = generator.predict(noise)

        d_loss_real = discriminator.train_on_batch(real_imgs, valid)
        d_loss_fake = discriminator.train_on_batch(gen_imgs, fake)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        # Train Generator
        noise = np.random.normal(0, 1, (batch_size, latent_dim))
        g_loss = gan.train_on_batch(noise, valid)

        # Print the progress
        print(f"{epoch} [D loss: {d_loss[0]} | D acc.: {100*d_loss[1]}] [G loss: {g_loss}]")

        # Save generated images at intervals
        if epoch % save_interval == 0:
            save_imgs(epoch)

# Save generated images function
def save_imgs(epoch):
    r, c = 5, 5
    noise = np.random.normal(0, 1, (r * c, latent_dim))
    gen_imgs = generator.predict(noise)

    gen_imgs = 0.5 * gen_imgs + 0.5  # Rescale to [0, 1]

    fig, axs = plt.subplots(r, c)
    cnt = 0
    for i in range(r):
        for j in range(c):
            axs[i, j].imshow(gen_imgs[cnt])
            axs[i, j].axis('off')
            cnt += 1
    fig.savefig(f"images/gan_{epoch}.png")
    plt.close()

# Train the GAN
train(epochs, batch_size, save_interval)


2/2 [==============================] - 1s 5ms/step
0 [D loss: 19.49143624305725 | D acc.: 27.34375] [G loss: 0.678414523601532]
2/2 [==============================] - 0s 3ms/step
1 [D loss: 0.5675987601280212 | D acc.: 50.0] [G loss: 0.3758172392845154]
2/2 [==============================] - 0s 4ms/step
2 [D loss: 0.8926606178283691 | D acc.: 50.0] [G loss: 0.19813640415668488]
2/2 [==============================] - 0s 3ms/step
3 [D loss: 1.3333871364593506 | D acc.: 50.0] [G loss: 0.09129853546619415]
2/2 [==============================] - 0s 3ms/step
4 [D loss: 1.7216696739196777 | D acc.: 50.0] [G loss: 0.05150698125362396]
2/2 [==============================] - 0s 3ms/step
5 [D loss: 2.13161563873291 | D acc.: 50.0] [G loss: 0.020003803074359894]
2/2 [==============================] - 0s 3ms/step
6 [D loss: 2.5063114166259766 | D acc.: 50.0] [G loss: 0.014484655112028122]
2/2 [==============================] - 0s 3ms/step
7 [D loss: 2.545095443725586 | D acc.: 50.0] [G loss: 0.0103

In [9]:
# import numpy as np
# import matplotlib.pyplot as plt

# # Define the latent dimension
# latent_dim = 10

# # Load the trained generator model (if not already in memory)
# # Make sure to load the appropriate weights if they were saved during training
# # generator.load_weights('path_to_generator_weights.h5')

# # Function to generate and save new images
# def generate_and_save_images(generator, num_images, latent_dim, save_path='generated_images'):
#     noise = np.random.normal(0, 1, (num_images, latent_dim))
#     gen_imgs = generator.predict(noise)
    
#     gen_imgs = 0.5 * gen_imgs + 0.5  # Rescale to [0, 1]
    
#     r, c = int(np.sqrt(num_images)), int(np.sqrt(num_images))
#     fig, axs = plt.subplots(r, c)
#     cnt = 0
#     for i in range(r):
#         for j in range(c):
#             axs[i, j].imshow(gen_imgs[cnt])
#             axs[i, j].axis('off')
#             cnt += 1
#     fig.savefig(f"{save_path}.png")
#     plt.close()

# # Generate and save new images
# generate_and_save_images(generator, num_images=25, latent_dim=latent_dim)


In [1]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import Xception
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
import face_recognition

In [2]:
def video_to_frames(video_path, output_dir):
    cap = cv2.VideoCapture(video_path)
    frame_count = 0
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frame_path = os.path.join(output_dir, f"frame_{frame_count}.jpg")
        cv2.imwrite(frame_path, frame)
        frame_count += 1
    
    cap.release()


In [7]:
# 2. Face Detection and Extraction
def extract_faces_from_frames(frames_dir, faces_dir, label):
    label_dir = os.path.join(faces_dir, label)
    os.makedirs(label_dir, exist_ok=True)
    
    for frame_name in os.listdir(frames_dir):
        frame_path = os.path.join(frames_dir, frame_name)
        image = cv2.imread(frame_path)
        face_locations = face_recognition.face_locations(image)

        for i, face_location in enumerate(face_locations):
            top, right, bottom, left = face_location
            face_image = image[top:bottom, left:right]
            face_path = os.path.join(label_dir, f"{frame_name}_face_{i}.jpg")
            cv2.imwrite(face_path, face_image)


In [9]:
# 3. Process All Videos in Dataset
def process_dataset(dataset_dir, faces_dir):
    categories = ['Real', 'Fake']
    
    for category in categories:
        category_dir = os.path.join(dataset_dir, category)
        temp_frames_dir = os.path.join(faces_dir, 'temp_frames')
        os.makedirs(temp_frames_dir, exist_ok=True)
        
        for video_name in os.listdir(category_dir):
            video_path = os.path.join(category_dir, video_name)
            video_to_frames(video_path, temp_frames_dir)
            extract_faces_from_frames(temp_frames_dir, faces_dir, label=category)
            # Clean up temporary frames directory
            for file in os.listdir(temp_frames_dir):
                os.remove(os.path.join(temp_frames_dir, file))

def create_data_generator(faces_dir, batch_size=32, img_size=(224, 224)):
    datagen = ImageDataGenerator(rescale=1.0/255.0, validation_split=0.2)
    train_gen = datagen.flow_from_directory(
        faces_dir,
        target_size=img_size,
        batch_size=batch_size,
        class_mode='binary',
        subset='training'
    )
    val_gen = datagen.flow_from_directory(
        faces_dir,
        target_size=img_size,
        batch_size=batch_size,
        class_mode='binary',
        subset='validation'
    )
    return train_gen, val_gen

In [5]:
def build_model(input_shape=(224, 224, 3)):
    base_model = Xception(weights='imagenet', include_top=False, input_shape=input_shape)
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    predictions = Dense(1, activation='sigmoid')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    
    # Freeze the base model layers
    for layer in base_model.layers:
        layer.trainable = False
    
    return model

# 5. Compile and Train the Model on GPU
def train_model(model, train_gen, val_gen, epochs=10):
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    # Use GPU if available
    with tf.device('/GPU:0'):
        model.fit(train_gen, validation_data=val_gen, epochs=epochs)

In [6]:
# 6. Save the Trained Model
def save_model(model, model_path):
    model.save(model_path)

# 7. Inference on New Videos
def predict_on_video(model, video_path, temp_dir, img_size=(224, 224)):
    video_to_frames(video_path, temp_dir)
    extract_faces_from_frames(temp_dir, temp_dir)
    
    predictions = []
    for face_name in os.listdir(temp_dir):
        face_path = os.path.join(temp_dir, face_name)
        image = cv2.imread(face_path)
        image = cv2.resize(image, img_size)
        image = np.expand_dims(image, axis=0) / 255.0
        prediction = model.predict(image)
        predictions.append(prediction)
    
    return np.mean(predictions) > 0.5  # Return True for deepfake, False otherwise


In [10]:
dataset_dir = './dataset'
faces_dir = './extracted_faces'
model_save_path = 'deepfake_detection_model.h5'

# Process the dataset (Extract faces from videos)
process_dataset(dataset_dir, faces_dir)

# Prepare data generators
train_gen, val_gen = create_data_generator(faces_dir)

KeyboardInterrupt: 

In [ ]:
# Build and train the model
model = build_model()
train_model(model, train_gen, val_gen)

In [ ]:

# Save the model
save_model(model, model_save_path)

# Predict on a new video
is_deepfake = predict_on_video(model, 'path/to/new_video.mp4', 'path/to/temp_frames')
print(f"Deepfake detected: {is_deepfake}")

Total no of videos : 200
['./dataset/Fake\\id0_id20_0007.mp4', './dataset/Fake\\id0_id23_0002.mp4', './dataset/Fake\\id0_id26_0007.mp4', './dataset/Fake\\id0_id2_0008.mp4', './dataset/Fake\\id0_id3_0009.mp4', './dataset/Fake\\id0_id4_0003.mp4', './dataset/Fake\\id0_id4_0004.mp4', './dataset/Fake\\id0_id9_0009.mp4', './dataset/Fake\\id10_id7_0002.mp4', './dataset/Fake\\id11_id7_0001.mp4', './dataset/Fake\\id16_id35_0003.mp4', './dataset/Fake\\id16_id37_0010.mp4', './dataset/Fake\\id17_id30_0005.mp4', './dataset/Fake\\id17_id35_0009.mp4', './dataset/Fake\\id19_id23_0004.mp4', './dataset/Fake\\id1_id21_0000.mp4', './dataset/Fake\\id1_id28_0006.mp4', './dataset/Fake\\id20_id0_0006.mp4', './dataset/Fake\\id20_id16_0007.mp4', './dataset/Fake\\id20_id3_0009.mp4', './dataset/Fake\\id21_id19_0007.mp4', './dataset/Fake\\id21_id23_0007.mp4', './dataset/Fake\\id21_id26_0005.mp4', './dataset/Fake\\id21_id26_0009.mp4', './dataset/Fake\\id21_id37_0007.mp4', './dataset/Fake\\id22_id19_0006.mp4', './da

frames are  [630, 457, 469, 477, 332, 320, 302, 458, 519, 404, 339, 317, 467, 333, 321, 555, 459, 332, 316, 479, 349, 325, 332, 366, 298, 454, 551, 457, 326, 310, 445, 483, 335, 377, 504, 485, 468, 344, 503, 557, 499, 342, 441, 323, 306, 314, 449, 456, 585, 485, 334, 322, 323, 470, 470, 314, 267, 464, 333, 235, 464, 380, 333, 376, 437, 228, 186, 485, 298, 501, 416, 335, 529, 470, 520, 398, 458, 505, 610, 317, 452, 397, 350, 480, 333, 516, 454, 383, 470, 530, 494, 404, 324, 458, 376, 441, 327, 312, 344, 465, 314, 466, 517, 527, 410, 475, 309, 467, 474, 305, 551, 267, 335, 317, 464, 401, 328, 489, 492, 319, 373, 363, 362, 448, 323, 314, 459, 302, 312, 342, 582, 489, 249, 366, 331, 555, 259, 479, 280, 331, 479, 438, 437, 334, 370, 505, 451, 427, 303, 333, 441, 497, 469, 464, 342, 324, 449, 400, 306, 347, 334, 487, 365, 353, 311, 317, 291, 330, 451, 520, 380, 520, 315, 305, 307, 454, 313, 381, 342, 334, 330, 491, 582, 404, 414, 485, 452, 335, 451, 165, 314, 308, 473, 530, 570, 328, 473, 49

In [10]:
# !pip install remotezip tqdm opencv-python einops

In [11]:
import tqdm
import random
import pathlib
import itertools
import collections

import cv2
import einops
import numpy as np
import remotezip as rz
import seaborn as sns
import matplotlib.pyplot as plt

import tensorflow as tf
import keras
from keras import layers

In [12]:
# Define the dimensions of one frame in the set of frames created
HEIGHT = 224
WIDTH = 224

In [13]:
class Conv2Plus1D(keras.layers.Layer):
  def __init__(self, filters, kernel_size, padding):
    super().__init__()
    self.seq = keras.Sequential([  
        layers.Conv3D(filters=filters,
                      kernel_size=(1, kernel_size[1], kernel_size[2]),
                      padding=padding),
        layers.Conv3D(filters=filters, 
                      kernel_size=(kernel_size[0], 1, 1),
                      padding=padding)
        ])

  def call(self, x):
    return self.seq(x)

In [14]:
class ResidualMain(keras.layers.Layer):
  """
    Residual block of the model with convolution, layer normalization, and the
    activation function, ReLU.
  """
  def __init__(self, filters, kernel_size):
    super().__init__()
    self.seq = keras.Sequential([
        Conv2Plus1D(filters=filters,
                    kernel_size=kernel_size,
                    padding='same'),
        layers.LayerNormalization(),
        layers.ReLU(),
        Conv2Plus1D(filters=filters, 
                    kernel_size=kernel_size,
                    padding='same'),
        layers.LayerNormalization()
    ])

  def call(self, x):
    return self.seq(x)

In [15]:
class Project(keras.layers.Layer):
  """
    Project certain dimensions of the tensor as the data is passed through different 
    sized filters and downsampled. 
  """
  def __init__(self, units):
    super().__init__()
    self.seq = keras.Sequential([
        layers.Dense(units),
        layers.LayerNormalization()
    ])

  def call(self, x):
    return self.seq(x)

In [16]:
def add_residual_block(input, filters, kernel_size):
  """
    Add residual blocks to the model. If the last dimensions of the input data
    and filter size does not match, project it such that last dimension matches.
  """
  out = ResidualMain(filters, 
                     kernel_size)(input)

  res = input
  # Using the Keras functional APIs, project the last dimension of the tensor to
  # match the new filter size
  if out.shape[-1] != input.shape[-1]:
    res = Project(out.shape[-1])(res)

  return layers.add([res, out])

In [17]:
class ResizeVideo(keras.layers.Layer):
  def __init__(self, height, width):
    super().__init__()
    self.height = height
    self.width = width
    self.resizing_layer = layers.Resizing(self.height, self.width)

  def call(self, video):
    """
      Use the einops library to resize the tensor.  

      Args:
        video: Tensor representation of the video, in the form of a set of frames.

      Return:
        A downsampled size of the video according to the new height and width it should be resized to.
    """
    # b stands for batch size, t stands for time, h stands for height, 
    # w stands for width, and c stands for the number of channels.
    old_shape = einops.parse_shape(video, 'b t h w c')
    images = einops.rearrange(video, 'b t h w c -> (b t) h w c')
    images = self.resizing_layer(images)
    videos = einops.rearrange(
        images, '(b t) h w c -> b t h w c',
        t = old_shape['t'])
    return videos

In [42]:
input_shape = (None, 30, HEIGHT, WIDTH, 3)
input = layers.Input(shape=(input_shape[1:]))
x = input

x = Conv2Plus1D(filters=16, kernel_size=(3, 7, 7), padding='same')(x)
x = layers.BatchNormalization()(x)
x = layers.ReLU()(x)
x = ResizeVideo(HEIGHT // 2, WIDTH // 2)(x)

# Block 1
x = add_residual_block(x, 16, (3, 3, 3))
x = ResizeVideo(HEIGHT // 4, WIDTH // 4)(x)

# Block 2
x = add_residual_block(x, 32, (3, 3, 3))
x = ResizeVideo(HEIGHT // 8, WIDTH // 8)(x)

# Block 3
x = add_residual_block(x, 64, (3, 3, 3))
x = ResizeVideo(HEIGHT // 16, WIDTH // 16)(x)

# Block 4
x = add_residual_block(x, 128, (3, 3, 3))

x = layers.GlobalAveragePooling3D()(x)
x = layers.Flatten()(x)
x = layers.Dense(10)(x)

model = keras.Model(input, x)

In [43]:
from tensorflow.keras.preprocessing import image_dataset_from_directory

real_videos_path = 'dataset/Real'
fake_videos_path = 'dataset/Fake'
frames_per_video = 30  # Number of frames to extract per video

def load_video(video_path, max_frames=frames_per_video):
    cap = cv2.VideoCapture(video_path)
    frames = []
    try:
        while len(frames) < max_frames:
            ret, frame = cap.read()
            if not ret:
                break
            frame = cv2.resize(frame, (WIDTH, HEIGHT))
            frames.append(frame)
        # Pad with last frame if not enough frames
        while len(frames) < max_frames:
            frames.append(frames[-1])
    finally:
        cap.release()
    return np.array(frames)


real_videos = [os.path.join(real_videos_path, f) for f in os.listdir(real_videos_path) if f.endswith('.mp4')]
fake_videos = [os.path.join(fake_videos_path, f) for f in os.listdir(fake_videos_path) if f.endswith('.mp4')]

real_labels = [0] * len(real_videos)  # Assuming label 0 for Real
fake_labels = [1] * len(fake_videos)  # Assuming label 1 for Fake

videos = real_videos + fake_videos
labels = real_labels + fake_labels


In [44]:
from sklearn.model_selection import train_test_split
train_videos, val_videos, train_labels, val_labels = train_test_split(videos, labels, test_size=0.2, random_state=123)

In [45]:
def video_generator(video_paths, labels):
    for video_path, label in zip(video_paths, labels):
        frames = load_video(video_path)
        yield frames, label

train_dataset = tf.data.Dataset.from_generator(
    lambda: video_generator(train_videos, train_labels),
    output_signature=(
        tf.TensorSpec(shape=(frames_per_video, HEIGHT, WIDTH, 3), dtype=tf.float32),
        tf.TensorSpec(shape=(), dtype=tf.int32))
)

val_dataset = tf.data.Dataset.from_generator(
    lambda: video_generator(val_videos, val_labels),
    output_signature=(
        tf.TensorSpec(shape=(frames_per_video, HEIGHT, WIDTH, 3), dtype=tf.float32),
        tf.TensorSpec(shape=(), dtype=tf.int32))
)


In [46]:


batch_size = 8
AUTOTUNE = tf.data.AUTOTUNE 

train_dataset = train_dataset.batch(batch_size).prefetch(buffer_size=AUTOTUNE)
val_dataset = val_dataset.batch(batch_size).prefetch(buffer_size=AUTOTUNE)


In [54]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    print("Name:", gpu.name, "  Type:", gpu.device_type)

Name: /physical_device:GPU:0   Type: GPU


In [55]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 6833060629529138832
 xla_global_id: -1,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 2243113780
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 7527072639593885194
 physical_device_desc: "device: 0, name: NVIDIA GeForce GTX 1650, pci bus id: 0000:01:00.0, compute capability: 7.5"
 xla_global_id: 416903419]

In [47]:
frames, label = next(iter(train_dataset))
model.build(frames)

In [48]:
# keras.utils.plot_model(model, expand_nested=True, dpi=60, show_shapes=True)

In [49]:
model.compile(loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True),  optimizer = keras.optimizers.Adam(learning_rate = 0.0001), metrics = ['accuracy'])

In [50]:
history = model.fit(x = train_dataset,
                    epochs = 50, 
                    validation_data = val_dataset)

Epoch 1/50


ResourceExhaustedError: Graph execution error:

Detected at node 'model_1/batch_normalization_1/FusedBatchNormV3' defined at (most recent call last):
    File "c:\Users\udits\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "c:\Users\udits\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "C:\Users\udits\AppData\Roaming\Python\Python310\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "C:\Users\udits\AppData\Roaming\Python\Python310\site-packages\traitlets\config\application.py", line 1043, in launch_instance
      app.start()
    File "C:\Users\udits\AppData\Roaming\Python\Python310\site-packages\ipykernel\kernelapp.py", line 712, in start
      self.io_loop.start()
    File "C:\Users\udits\AppData\Roaming\Python\Python310\site-packages\tornado\platform\asyncio.py", line 215

In [1]:
import cv2
import os
import face_recognition

# Function to process the video, extract frames, detect faces, and save them simultaneously
def process_video(video_path, faces_dir):
    if not os.path.exists(faces_dir):
        os.makedirs(faces_dir)
    
    cap = cv2.VideoCapture(video_path)
    frame_count = 0
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        # Detect faces in the current frame
        face_locations = face_recognition.face_locations(frame)

        for i, face_location in enumerate(face_locations):
            top, right, bottom, left = face_location
            face_image = frame[top:bottom, left:right]
            face_path = os.path.join(faces_dir, f"frame_{frame_count}_face_{i}.jpg")
            cv2.imwrite(face_path, face_image)
        
        frame_count += 1
    
    cap.release()

# # Example usage:
# video_path = '/dataset/Fake/*.mp4' 
# faces_dir = './faces'

# process_video(video_path, faces_dir)